In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [3]:
image_size=256
batch_size=32
channels=3

In [4]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "plant",
    shuffle=True,
    image_size=(image_size,image_size),
    batch_size=batch_size,


)

Found 16011 files belonging to 10 classes.


In [5]:
class_names=dataset.class_names
class_names


['Tomato_Bacterial_spot',
 'Tomato_Early_blight',
 'Tomato_Late_blight',
 'Tomato_Leaf_Mold',
 'Tomato_Septoria_leaf_spot',
 'Tomato_Spider_mites_Two_spotted_spider_mite',
 'Tomato__Target_Spot',
 'Tomato__Tomato_YellowLeaf__Curl_Virus',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']

In [6]:
def get_dataset_partition_tf(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=1000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)

    return train_ds,val_ds,test_ds

In [7]:
train_ds,val_ds,test_ds=get_dataset_partition_tf(dataset)


In [8]:
len(train_ds)

400

In [11]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [12]:
resizing_and_rescaling=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(image_size,image_size),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [13]:
data_argumentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [22]:
input_shape=(batch_size,image_size,image_size,channels)
model=models.Sequential([
    
 
    resizing_and_rescaling,
    
    data_argumentation,
    
    
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='softmax'),
])
model.build(input_shape=input_shape)

In [23]:
model.compile(
    
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [25]:
history=model.fit(
    train_ds,
    epochs=10,
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds,

)

Epoch 1/10
400/400 [==============================] - 668s 2s/step - loss: 1.3788 - accuracy: 0.5044 - val_loss: 1.2433 - val_accuracy: 0.5850
Epoch 2/10
400/400 [==============================] - 623s 2s/step - loss: 0.7268 - accuracy: 0.7432 - val_loss: 0.9347 - val_accuracy: 0.6831
Epoch 3/10
400/400 [==============================] - 620s 2s/step - loss: 0.5766 - accuracy: 0.7977 - val_loss: 0.9348 - val_accuracy: 0.7038
Epoch 4/10
400/400 [==============================] - 620s 2s/step - loss: 0.4467 - accuracy: 0.8462 - val_loss: 0.9123 - val_accuracy: 0.7019
Epoch 5/10
400/400 [==============================] - 602s 2s/step - loss: 0.3988 - accuracy: 0.8623 - val_loss: 0.9581 - val_accuracy: 0.6875
Epoch 6/10
400/400 [==============================] - 596s 1s/step - loss: 0.3582 - accuracy: 0.8762 - val_loss: 0.9116 - val_accuracy: 0.7650
Epoch 7/10
400/400 [==============================] - 597s 1s/step - loss: 0.3134 - accuracy: 0.8905 - val_loss: 0.5893 - val_accuracy: 0.8331

In [26]:
score=model.evaluate(test_ds)

51/51 [==============================] - 34s 335ms/step - loss: 0.6445 - accuracy: 0.7911


In [27]:
score


[0.6445393562316895, 0.7910539507865906]